In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse

In [2]:
import mlflow
import mlflow.sklearn

In [4]:
%matplotlib inline

In [5]:
def eval_metrics(actual, pred):
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return mse, rmse, mae, r2


In [6]:
# Load the diabetes dataset
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]


In [7]:
# Create linear regression object
regr = linear_model.LinearRegression()

with mlflow.start_run():
    # Train the model using the training sets
    regr.fit(diabetes_X_train, diabetes_y_train)
    
    # Make predictions using the testing set
    diabetes_y_pred = regr.predict(diabetes_X_test)
    
    (mse, rmse, mae, r2) = eval_metrics(diabetes_y_test, diabetes_y_pred)
    
    #print the values
    
    # The coefficients
    print('Coefficients: \n', regr.coef_)
    
    # The mean squared error
    print('Mean squared error: %.2f'
      % mse)

    # The root mean squared error
    print('Rroot Mean squared error: %.2f'
      % rmse)
    
    # The mean absolute error
    print('Mean absolute error: %.2f'
      % mae)
    
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
      % r2)
    
    mlflow.log_param("coefficient", regr.coef_)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("mae", mae)
    mlflow.log_metric("r2", r2)
    
    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":
        
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(regr, "model", registered_model_name="LinearRegressionModel")
    
    else:
        mlflow.sklearn.log_model(regr, "model")

Coefficients: 
 [938.23786125]
Mean squared error: 2548.07
Rroot Mean squared error: 50.48
Mean absolute error: 41.23
Coefficient of determination: 0.47
